Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in `1_notmnist.ipynb`.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [11]:
alpha = 0.01
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  loss = tf.reduce_mean(loss + alpha*tf.nn.l2_loss(weights))
  
  # Optimizer.  
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [12]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 45.638031
Minibatch accuracy: 9.4%
Validation accuracy: 13.6%
Minibatch loss at step 500: 0.765295
Minibatch accuracy: 88.3%
Validation accuracy: 82.0%
Minibatch loss at step 1000: 0.597345
Minibatch accuracy: 87.5%
Validation accuracy: 81.0%
Minibatch loss at step 1500: 0.778516
Minibatch accuracy: 82.0%
Validation accuracy: 80.9%
Minibatch loss at step 2000: 0.815655
Minibatch accuracy: 80.5%
Validation accuracy: 82.1%
Minibatch loss at step 2500: 0.830631
Minibatch accuracy: 78.9%
Validation accuracy: 81.2%
Minibatch loss at step 3000: 0.600832
Minibatch accuracy: 82.8%
Validation accuracy: 81.9%
Test accuracy: 88.1%


In [13]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  W1 = tf.Variable(tf.truncated_normal([image_size * image_size, 1024] ,stddev=0.1))
  B1 = tf.Variable(tf.zeros([1024]))

  W2 = tf.Variable(tf.truncated_normal([1024, num_labels]))
  B2 = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, W1) + B1
  Y1 = tf.nn.relu(logits)
  logits2 = tf.matmul(Y1, W2) + B2
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits2)) 
  loss += + alpha*tf.nn.l2_loss(W1) + + alpha*tf.nn.l2_loss(W2)
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  train_prediction = tf.nn.softmax(logits2)
  valid_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, W1) + B1), W2) + B2)
  test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, W1) + B1), W2) + B2)

In [14]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 109.285492
Minibatch accuracy: 4.7%
Validation accuracy: 37.0%
Minibatch loss at step 500: 1.006268
Minibatch accuracy: 91.4%
Validation accuracy: 84.2%
Minibatch loss at step 1000: 0.587496
Minibatch accuracy: 88.3%
Validation accuracy: 83.4%
Minibatch loss at step 1500: 0.701778
Minibatch accuracy: 85.2%
Validation accuracy: 84.2%
Minibatch loss at step 2000: 0.722973
Minibatch accuracy: 83.6%
Validation accuracy: 84.6%
Minibatch loss at step 2500: 0.721177
Minibatch accuracy: 83.6%
Validation accuracy: 84.5%
Minibatch loss at step 3000: 0.600424
Minibatch accuracy: 85.2%
Validation accuracy: 84.7%
Test accuracy: 90.5%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [15]:
train_dataset_small = train_dataset[:500, :]
train_labels_small = train_labels[:500]
num_steps = 3001 

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels_small.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset_small[offset:(offset + batch_size), :]
    batch_labels = train_labels_small[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 106.526802
Minibatch accuracy: 3.1%
Validation accuracy: 38.6%
Minibatch loss at step 500: 0.747726
Minibatch accuracy: 100.0%
Validation accuracy: 79.2%
Minibatch loss at step 1000: 0.316662
Minibatch accuracy: 100.0%
Validation accuracy: 79.2%
Minibatch loss at step 1500: 0.279229
Minibatch accuracy: 100.0%
Validation accuracy: 79.2%
Minibatch loss at step 2000: 0.266537
Minibatch accuracy: 100.0%
Validation accuracy: 79.0%
Minibatch loss at step 2500: 0.262954
Minibatch accuracy: 100.0%
Validation accuracy: 79.0%
Minibatch loss at step 3000: 0.260373
Minibatch accuracy: 100.0%
Validation accuracy: 78.8%
Test accuracy: 86.1%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [16]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  pkeep = tf.placeholder(tf.float32)
    
  # Variables.
  W1 = tf.Variable(tf.truncated_normal([image_size * image_size, 1024] ,stddev=0.1))
  B1 = tf.Variable(tf.zeros([1024]))

  W2 = tf.Variable(tf.truncated_normal([1024, num_labels]))
  B2 = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits1 = tf.matmul(tf_train_dataset, W1) + B1
  Y1 = tf.nn.relu(logits1)
  Y1d = tf.nn.dropout(Y1, pkeep)

  logits2 = tf.matmul(Y1d, W2) + B2
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits2))
  # Loss function with L2 Regularization with beta=0.01
  l2_regularization = tf.nn.l2_loss(W1) + tf.nn.l2_loss(W2)
  loss = tf.reduce_mean(loss + alpha * l2_regularization)
  
    # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  train_prediction = tf.nn.softmax(logits2)
  valid_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, W1) + B1), W2) + B2)
  test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, W1) + B1), W2) + B2)

In [17]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run() #tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, pkeep: 0.5}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 121.678795
Minibatch accuracy: 6.2%
Validation accuracy: 41.0%
Minibatch loss at step 500: 1.099785
Minibatch accuracy: 90.6%
Validation accuracy: 83.7%
Minibatch loss at step 1000: 0.670427
Minibatch accuracy: 86.7%
Validation accuracy: 82.9%
Minibatch loss at step 1500: 0.816153
Minibatch accuracy: 82.8%
Validation accuracy: 83.7%
Minibatch loss at step 2000: 0.766258
Minibatch accuracy: 82.8%
Validation accuracy: 83.9%
Minibatch loss at step 2500: 0.812633
Minibatch accuracy: 81.2%
Validation accuracy: 84.1%
Minibatch loss at step 3000: 0.685650
Minibatch accuracy: 82.0%
Validation accuracy: 84.2%
Test accuracy: 90.0%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [47]:
import math as math


alpha = 0.002
batch_size = 128

n_layer_1 = 2048
n_layer_2 = 1024
n_layer_3 = 512
n_layer_4 = 256

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  pkeep = tf.placeholder(tf.float32)
    
  # Variables.
  W1 = tf.Variable(tf.truncated_normal([image_size * image_size, n_layer_1] ,stddev=math.sqrt(2.0/(image_size*image_size))))
  B1 = tf.Variable(tf.zeros([n_layer_1]))

  W2 = tf.Variable(tf.truncated_normal([n_layer_1, n_layer_2], stddev=math.sqrt(2.0/n_layer_1)))
  B2 = tf.Variable(tf.zeros([n_layer_2]))
  
  W3 = tf.Variable(tf.truncated_normal([n_layer_2, n_layer_3], stddev=math.sqrt(2.0/n_layer_2)))
  B3 = tf.Variable(tf.zeros([n_layer_3]))

  W4 = tf.Variable(tf.truncated_normal([n_layer_3, n_layer_4], stddev=math.sqrt(2.0/n_layer_3)))
  B4 = tf.Variable(tf.zeros([n_layer_4]))
  
  W5 = tf.Variable(tf.truncated_normal([n_layer_4, num_labels], stddev=math.sqrt(2.0/n_layer_4)))
  B5 = tf.Variable(tf.zeros([num_labels]))
                   
  # Training computation.
  Y1 = tf.nn.relu(tf.matmul(tf_train_dataset, W1) + B1)
  Y1d = tf.nn.dropout(Y1, pkeep)
                   
  Y2 = tf.nn.relu(tf.matmul(Y1d, W2) + B2)
  Y2d = tf.nn.dropout(Y2, pkeep)
                   
  Y3 = tf.nn.relu(tf.matmul(Y2d, W3) + B3)
  Y3d = tf.nn.dropout(Y3, pkeep)     
                                  
  Y4 = tf.nn.relu(tf.matmul(Y3d, W4) + B4)
  Y4d = tf.nn.dropout(Y4, pkeep)
  
  logits = tf.matmul(Y4d, W5) + B5
  Y5 = tf.nn.softmax(logits)  
                   
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  l2_regularization = tf.nn.l2_loss(W1) + tf.nn.l2_loss(W2) + tf.nn.l2_loss(W3) + \
    tf.nn.l2_loss(W4) + tf.nn.l2_loss(W5)
  loss = tf.reduce_mean(loss + alpha * l2_regularization)
    
  # Optimizer
  global_step = tf.Variable(0)  # count the number of steps taken.
  # the learning rate is: # 0.0001 + 0.003 * (1/e)^(step/2000)), i.e. exponential decay from 0.003->0.0001
  lr =  0.0001 + tf.train.exponential_decay(0.5, global_step, 5000, 1/math.e)
  #learning_rate = tf.train.exponential_decay(0.5, global_step, 100000, 0.96, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(lr).minimize(loss, global_step=global_step)


  # Predictions for the training
  train_prediction = tf.nn.softmax(logits)

  # Predictions for validation 
  valid_logits_1 = tf.matmul(tf_valid_dataset, W1) + B1
  valid_relu_1 = tf.nn.relu(valid_logits_1)
    
  valid_logits_2 = tf.matmul(valid_relu_1, W2) + B2
  valid_relu_2 = tf.nn.relu(valid_logits_2)
    
  valid_logits_3 = tf.matmul(valid_relu_2, W3) + B3
  valid_relu_3 = tf.nn.relu(valid_logits_3)
    
  valid_logits_4 = tf.matmul(valid_relu_3, W4) + B4
  valid_relu_4 = tf.nn.relu(valid_logits_4)
  
  valid_logits_5 = tf.matmul(valid_relu_4, W5) + B5    
  valid_prediction = tf.nn.softmax(valid_logits_5)
    
  # Predictions for test
  test_logits_1 = tf.matmul(tf_test_dataset, W1) + B1
  test_relu_1 = tf.nn.relu(test_logits_1)
    
  test_logits_2 = tf.matmul(test_relu_1, W2) + B2
  test_relu_2 = tf.nn.relu(test_logits_2)
   
  test_logits_3 = tf.matmul(test_relu_2, W3) + B3
  test_relu_3 = tf.nn.relu(test_logits_3)
    
  test_logits_4 = tf.matmul(test_relu_3, W4) + B4
  test_relu_4 = tf.nn.relu(test_logits_4)
    
  test_logits_5 = tf.matmul(test_relu_4, W5) + B5
  test_prediction = tf.nn.softmax(test_logits_5)
  

In [48]:
num_steps = 10001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, pkeep : 0.5}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step {}: {}".format(step, l))
            print("Minibatch accuracy: {:.1f}".format(accuracy(predictions, batch_labels)))
            print("Validation accuracy: {:.1f}".format(accuracy(valid_prediction.eval(), valid_labels)))
    print("Test accuracy: {:.1f}".format(accuracy(test_prediction.eval(), test_labels)))

Initialized
Minibatch loss at step 0: 8.87366771697998
Minibatch accuracy: 12.5
Validation accuracy: 10.0
Minibatch loss at step 500: 2.911520481109619
Minibatch accuracy: 89.8
Validation accuracy: 84.5
Minibatch loss at step 1000: 1.5503265857696533
Minibatch accuracy: 91.4
Validation accuracy: 85.1
Minibatch loss at step 1500: 1.1114579439163208
Minibatch accuracy: 85.2
Validation accuracy: 85.1
Minibatch loss at step 2000: 0.9213698506355286
Minibatch accuracy: 84.4
Validation accuracy: 86.5
Minibatch loss at step 2500: 0.7825955152511597
Minibatch accuracy: 86.7
Validation accuracy: 87.1
Minibatch loss at step 3000: 0.6101261377334595
Minibatch accuracy: 88.3
Validation accuracy: 87.1
Minibatch loss at step 3500: 0.6805579662322998
Minibatch accuracy: 85.9
Validation accuracy: 87.7
Minibatch loss at step 4000: 0.6788914203643799
Minibatch accuracy: 85.2
Validation accuracy: 87.3
Minibatch loss at step 4500: 0.7911598086357117
Minibatch accuracy: 85.9
Validation accuracy: 88.0
Minib